Optimization reservs
- average for градиенный бустинг и линейную регрессию
- убрать время начала и номер матча
- сделать чтобы все матчи начинались с 0. Без отрицательных времен
- продумать с пустыми значениями - их очень много
- (Done 01 - use it) first_blood_time', 'first_blood_team' - алгоритм понимает что это связка???? только время - (+) для одной команды и (-) для другой   - Only worse!!!
- посчитать "производительность" команд каждую минуту. Может динамика поможет. 
- покуртить ручки алгоритмов - может поможет
- + добавить степени числовых признаков - выйти за линейность - **2 для градиентного бустинга
- сколько -1 комнат в списке
- (Done modify_dayaframe_03 - use it) попробовать просуммировать однотипные поля команды

Development convinience
 - small subset to test that code does not throw exceptions  - done
 - reasonable file naming 
 - self documented results
 - one button run


# functions

In [1]:
# on 1000 - best param =  {'C': 0.00080000000000000004} best score =  0.747402682882

# configuration variable
number_rows_to_use= -1 # 100   # -1 all records

# annoying warnings disabling

import warnings
warnings.filterwarnings("ignore")



import numpy as np
# global variables
heroes = [
    'r1_hero',
    'r2_hero',
    'r3_hero',
    'r4_hero',
    'r5_hero',
    'd1_hero',
    'd2_hero',
    'd3_hero',
    'd4_hero',
    'd5_hero'
]
fields_to_remove = [
    'r1_hero',
    'r2_hero',
    'r3_hero',
    'r4_hero',
    'r5_hero',
    'd1_hero',
    'd2_hero',
    'd3_hero',
    'd4_hero',
    'd5_hero',
    'lobby_type'
#    'start_time'
]

import datetime

def printtime():
    return datetime.datetime.utcnow()


# Удалите признаки, связанные с итогами матча (они помечены в описании данных как отсутствующие в тестовой выборке).
'''
Итог матча (данные поля отсутствуют в тестовой выборке, поскольку содержат информацию, выходящую за пределы первых 5 минут матча)
duration: длительность
radiant_win: 1, если победила команда Radiant, 0 — иначе
Состояние башен и барраков к концу матча (см. описание полей набора данных)
tower_status_radiant
tower_status_dire
barracks_status_radiant
barracks_status_dire
'''

# usage: match_info, y = remove_result_columns(features)
def remove_result_columns(the_features):
    print "remove_result_columns (Before): shape",the_features.shape
    rez_col_names = [
    'duration',
    'radiant_win',
    'tower_status_radiant',
    'tower_status_dire',
    'barracks_status_radiant',
    'barracks_status_dire'
    ]
    # res_info = features[rez_col_names]
    removed_rez = the_features.drop(rez_col_names, axis=1)
    print "remove_result_columns (After): shape",removed_rez.shape
    return removed_rez, the_features['radiant_win']


# remove initial heroes and lobbies columns
def remove_initial_heroes_and_lobbies(parX):
    print parX.shape, type(parX)
    X_categor_removed= parX.drop( heroes+ ['lobby_type'], axis=1)
    print X_categor_removed.shape, type(X_categor_removed)
    return X_categor_removed

'''
Замените пропуски на нули с помощью функции fillna(). 
На самом деле этот способ является предпочтительным для логистической регрессии, поскольку он позволит пропущенному 
значению не вносить никакого вклада в предсказание. Для деревьев часто лучшим вариантом оказывается замена пропуска 
на очень большое или очень маленькое значение — в этом случае при построении разбиения вершины можно будет отправить 
объекты с пропусками в отдельную ветвь дерева. Также есть и другие подходы — например, замена пропуска на среднее 
значение признака. Мы не требуем этого в задании, но при желании попробуйте разные подходы к обработке пропусков и 
сравните их между собой.
'''
# Замените пропуски на нули с помощью функции fillna().
def replace_skpped_values_with_0(theX):
    print 'Before'
    print 'NaNs =' ,np.count_nonzero(np.isnan(theX))
    print 'not NaNs =' ,np.count_nonzero(~np.isnan(theX))
    rez= theX.fillna(0)
    print 'after'
    print 'NaNs =' ,np.count_nonzero(np.isnan(rez))
    print 'not NaNs =' ,np.count_nonzero(~np.isnan(rez))
    return rez

'''
Среди признаков в выборке есть категориальные, которые мы использовали как числовые, что вряд ли является хорошей 
идеей. Категориальных признаков в этой задаче одиннадцать: lobby_type и r1_hero, r2_hero, ..., r5_hero, d1_hero, 
d2_hero, ..., d5_hero. Уберите их из выборки, и проведите кросс-валидацию для логистической регрессии на новой выборке 
с подбором лучшего параметра регуляризации. Изменилось ли качество? Чем вы можете это объяснить?
'''

# replace heroes with the bag of the words
def words_bag_of_heroes(theX):
    # calcule heroes range
    expected_num=113
    unique_heroes = np.unique(theX[heroes])
    if (max(unique_heroes)>expected_num):
        return 0 # throw exception here
    if (min(unique_heroes)<1):
        return 0 # throw exception here
    X_pick = np.zeros((theX.shape[0], expected_num))
    for i, match_id in enumerate(theX.index):
        for p in xrange(5):
            X_pick[i, theX.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
            X_pick[i, theX.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1
    return X_pick


# добавим мешок слов по комнате

'''id,name
-1,Invalid
0,Public matchmaking
1,Practice
2,Tournament
3,Tutorial
4,Co-op with bots
5,Team match
6,Solo Queue
7,Ranked
8,Solo Mid 1vs1
'''
        
#################################################################################################
def words_bag_of_rooms(theX):
    num_rooms = 10
    # check rooms range
    num_rooms_max = 8
    num_rooms_min = -1
    unique_lobbyes = np.unique(theX['lobby_type'])
    if (max(unique_lobbyes)>num_rooms_max):
        raise NameError(max(unique_lobbyes)+'too big')  
    if (min(unique_lobbyes) < num_rooms_min):
        raise NameError(max(unique_lobbyes)+'too small')
    # делаем мешок слов
    X_pick_room = np.zeros([theX.shape[0], num_rooms])   # !!! two (())
    for i, match_id in enumerate(theX.index):
        for p in xrange(10):
            X_pick_room[i, theX.ix[match_id, 'lobby_type']+1] = 1    
    return  X_pick_room       



# join X_pick heroes and rooms
def join_bags_rooms_and_heroes(parX_pick_heroes , parX_pick_rooms):
    print parX_pick_heroes.shape, parX_pick_rooms.shape
    rez=np.concatenate((parX_pick_heroes , parX_pick_rooms),  axis=1)
    print rez.shape
    return rez
    
    
# add word bag to the DataSet
from scipy.sparse import hstack
def add_word_bag(parDataSet, parX_pick):
    print parDataSet.shape, type(parDataSet)
    print parX_pick.shape, type(parX_pick)
    X_joined = hstack([parDataSet,parX_pick]).toarray()
    print X_joined.shape, type(X_joined)
    return X_joined 

'''
<b>Важно</b>: не забывайте, что линейные алгоритмы чувствительны к масштабу признаков! Может пригодиться sklearn.preprocessing.StandartScaler.
'''


from sklearn.preprocessing import StandardScaler
# usage
# ss = stdScaling_fit(X)
# X_std_scal = stdScaling_transform(X ,ss)
def stdScaling_fit(theX):
     return StandardScaler().fit(theX)

def stdScaling_transform(theX, stdScaler):
     return stdScaler.transform(theX)
    
    
# calculate C for logistic regression
from sklearn.grid_search import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import KFold
import numpy as np
import datetime


def calculate_logreg_best_param(X_par, Y_par):
    start_time = datetime.datetime.now()
    param_grid = {'C': [0.00000000001, 0.0001, 0.001 ,0.01, 0.1, 1, 10, 100, 1000, 100000000] }

    print "--- initial C calculation"
    print param_grid
    k_fold = KFold(len(Y_par), n_folds=5, shuffle=True, random_state=555)
    clf = LogisticRegression(penalty='l2', random_state=555, n_jobs=-1)
    gs_cv = GridSearchCV(clf, param_grid, scoring='roc_auc', cv=k_fold, n_jobs=-1)
    gs_cv.fit(X_par, Y_par)
    print "best param = ", gs_cv.best_params_, "best score = ", gs_cv.best_score_
    for score in gs_cv.grid_scores_:
        print score
    # fine tuning for 
    fine_grid = {'C': np.add(gs_cv.best_params_.get('C'), np.multiply(np.arange(-5, 6), gs_cv.best_params_.get('C')/10))}
    print "--- fine C tuning"
    print fine_grid
    gs_cv = GridSearchCV(clf, fine_grid, scoring='roc_auc', cv=k_fold)
    gs_cv.fit(X_par, Y_par)
    print "best param = ", gs_cv.best_params_, "best score = ", gs_cv.best_score_
    for score in gs_cv.grid_scores_:
        print score
    print 'Average time for single regression fit:', (datetime.datetime.now() - start_time)/(len(param_grid.get('C')) +len(fine_grid.get('C')))
    return gs_cv.best_params_


# GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import KFold
import datetime

# cross validation of GradientBoostingClassifier 
def calculate_scores_for_estimators(n_estimators, X, y, max_depth=3):
    print "calculate_scores_for_estimators: start ", printtime()
    start_time = datetime.datetime.now()
    clf = GradientBoostingClassifier(n_estimators=n_estimators,
        random_state=555, max_depth=max_depth
    )
    k_fold = KFold(len(y), n_folds=5, random_state=555, shuffle=True)
    local_scores = cross_val_score(clf, X, y, cv=k_fold, scoring='roc_auc', n_jobs=-1)
    print 'score_mean=',local_scores.mean(), 'scores_std=',local_scores.std(), \
        'n_estimators =', n_estimators, 'max_depth=', max_depth, 'Time:', datetime.datetime.now() - start_time
    print "calculate_scores_for_estimators: finish ", printtime()
    return local_scores.mean(), local_scores.std(), n_estimators, max_depth

def crossvalidate_estimators(arr_n_estimators, X, y, max_depth=3):
    print "crossvalidate_estimators: start ", printtime()
    rez_scores_local = list()
    for n_estimators in arr_n_estimators:    
        rez_scores_local.append(calculate_scores_for_estimators(n_estimators, X , y , max_depth=max_depth))    
    print "crossvalidate_estimators: finish ", printtime()    
    return rez_scores_local

def get_best_GradientBoostingClassifier(rez_scores_local):
    # get best classifier
    import sys
    print type(rez_scores_local)
    type(rez_scores_local[0])
    max_score=-sys.maxint - 1
    max_index="a"
    for index in range(len(rez_scores_local)):
        if (rez_scores_local[index][0]>max_score):
            max_index=index
            max_score=rez_scores_local[index][0]
            
        print rez_scores_local[index][0]
    best_n_estimators=rez_scores_local[max_index][2]
    best_max_depth=rez_scores_local[max_index][3]
    
    print 'max_score =', max_score, 'max_index=', max_index, \
        'best_n_estimators=', best_n_estimators, 'best_max_depth=', best_max_depth
    
    clf = GradientBoostingClassifier(n_estimators=best_n_estimators,
        random_state=555, max_depth=best_max_depth
    )
    return clf

# calculate predicted probabilities and store to file
#fileName = '/Volumes/fast64/My/MachineLearning/week07/Dota2_01/dota2-kaggle-01.csv'
def store_predicted_probability(clf, parX, parX_match_id, fileName):
    rez=clf.predict_proba(parX)[:, 1]
    print "Минимум предсказанных вероятностей =",min(rez)
    print "Максимум предсказанных вероятностей =", max(rez)
    check = {
        'match_id': parX_match_id.index, 
        'radiant_win': rez
    }

    check_dframe = pandas.DataFrame.from_dict(check)
    check_dframe.set_index('match_id')

    check_dframe.to_csv(fileName, index=False, columns=['match_id', 'radiant_win'])
    return check_dframe    


# prepare data for logistic regression
import pandas
def prepare_for_log_regr(fileName, isTest):
    # Считайте таблицу с признаками из файла features.csv с помощью кода, приведенного выше. 

    features = pandas.read_csv(fileName, index_col='match_id')
    if (number_rows_to_use>0):
        print "future trancated !!!!!!!!!!  to rows ", number_rows_to_use
        # http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sample.html
        features = features.sample(n=number_rows_to_use, random_state=555)

    features = modify_dayaframe_01(features)
    features = modify_dayaframe_03(features)
    features = modify_dayaframe_02(features)
    
    
    # print features.head()
    print features.info()
    
    if (isTest):        
        local_match_info = features
    else:    
        local_match_info, local_y = remove_result_columns(features)
   
    # calculate_importance(local_match_info, local_y)
        
    X_removed_heroes_rooms = remove_initial_heroes_and_lobbies(local_match_info)
    X = replace_skpped_values_with_0(X_removed_heroes_rooms)

    xxx_heroes= words_bag_of_heroes(local_match_info)
    print type(xxx_heroes)
    print xxx_heroes.shape

    xxx_rooms= words_bag_of_rooms(local_match_info)
    print type(xxx_rooms)
    print xxx_rooms.shape

    xxx_rooms_and_heroes=join_bags_rooms_and_heroes(xxx_heroes , xxx_rooms)

    X_bagged = add_word_bag(X,xxx_rooms_and_heroes)
    if (isTest):        
        return X_bagged, local_match_info
    else:    
        return X_bagged, local_y, local_match_info

    
# prepare data for gradient boosting
import pandas
def prepare_for_gradient_boosting(fileName, isTest):
    # Считайте таблицу с признаками из файла features.csv с помощью кода, приведенного выше. 

    features = pandas.read_csv(fileName, index_col='match_id')
    if (number_rows_to_use>0):
        print "future trancated !!!!!!!!!!  to rows ", number_rows_to_use
        # http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sample.html
        features = features.sample(n=number_rows_to_use, random_state=555)

    features = modify_dayaframe_01(features)
    features = modify_dayaframe_03(features)
    features = modify_dayaframe_02(features)
    
    
    # print features.head()
    print features.info()
    
    if (isTest):        
        local_match_info = features
    else:    
        local_match_info, local_y = remove_result_columns(features)
   
    # calculate_importance(local_match_info, local_y)
    X = replace_skpped_values_with_0(local_match_info)
    '''
    X_removed_heroes_rooms = remove_initial_heroes_and_lobbies(local_match_info)
    X = replace_skpped_values_with_0(X_removed_heroes_rooms)

    xxx_heroes= words_bag_of_heroes(local_match_info)
    print type(xxx_heroes)
    print xxx_heroes.shape

    xxx_rooms= words_bag_of_rooms(local_match_info)
    print type(xxx_rooms)
    print xxx_rooms.shape

    xxx_rooms_and_heroes=join_bags_rooms_and_heroes(xxx_heroes , xxx_rooms)

    X_bagged = add_word_bag(X,xxx_rooms_and_heroes)
    '''    
    if (isTest):        
        return X, local_match_info
    else:    
        return X, local_y, local_match_info
    
# sample code to append column to pandas.core.frame.DataFrame
def append_column_to_DataFrame():
    print x_bust_dataframe.shape
    x_bust_dataframe['new_field'] = x_bust_dataframe['r1_lh']**2

    print x_bust_dataframe['new_field'], x_bust_dataframe['r1_lh']
    print x_bust_dataframe.shape

    x_bust_dataframe['new_field_1'] = x_bust_dataframe.apply(lambda row: min([row['r1_lh'], row['r2_lh']])-row['r3_lh'], axis=1)
    print x_bust_dataframe.shape

# sample code to append column to numpy.ndarray
def append_column_to_ndarray():
    print x_bust.shape
    print x_bust[:,:1].shape
    # print x_bust[:,:1]
    new_x=np.append(x_bust, x_bust[:,:1]**2, 1)
    new_x.shape

'''
# Best usage is - add new field and remove 2 sources
# считает первую кровь со знаком в одном поле
# first_blood_time: игровое время первой крови
# first_blood_team: команда, совершившая первую кровь (0 — Radiant, 1 — Dire)
first_blood_player1: игрок, причастный к событию
first_blood_player2: второй игрок, причастный к событию
=== for log_regression: 5000
0.735882809137   – удалил все
0.736234081207 - удалил только 2 расчетных поля
# === for log_regression: 1000
# 0.716864897388- add 'first_blood_time_team', remove 'first_blood_team','first_blood_time', 'first_blood_player1','first_blood_player2'  
# 0.717485181078- add 'first_blood_time_team', remove 'first_blood_team','first_blood_time'  
# 0.716784645812 add 'first_blood_time_team'.
# 0.716864897388 - original   
# === for Gradient Boosting: 
# with this transformation 
# max_score = 0.697781663605 max_index= 0 best_n_estimators= 200 best_max_depth= 6
# original 
# max_score = 0.698382534947 max_index= 0 best_n_estimators= 200 best_max_depth= 6
'''
def modify_dayaframe_01(parDataFrame):
    print 'modify_dayaframe_01 (Before). parDataFrame', parDataFrame.shape
    parDataFrame['first_blood_time_team'] = \
        parDataFrame.apply(lambda row: row['first_blood_time']*(row['first_blood_team']*2-1) , axis=1)
    # print 'modify_dayaframe_01 (Middle). parDataFrame', parDataFrame.shape    
    # rezDataFrame=parDataFrame
    #rezDataFrame=parDataFrame
    rezDataFrame=parDataFrame.drop( ['first_blood_team','first_blood_time'], axis=1)
    # rezDataFrame=rezDataFrame.drop( ['first_blood_player1','first_blood_player2'], axis=1)
    print 'modify_dayaframe_01 (After). parDataFrame', rezDataFrame.shape
    return rezDataFrame

# add pow(x,) fields
# === for log_regression (1000 records)
# 0.729154975126 - original
# 0.736179073034 ** 2 
# 0.743773109605 ** 3
# 0.743972802269 ** 4
# === for log_regression:
# 0.714259140933add math.sqrt and remove original gold - make it even worse
# 0.721677111535 add **2 and remove original gold - make it even worse

# 0.729154975126 with function - added **2 - worse than original
# 0.716864897388 - original   (Better !!!!!!!!!!!)
# === for Gradient Boosting: 
# with this **2 and removed original
# score_mean= 0.699060828574 scores_std= 0.0242692498252 n_estimators = 200 max_depth= 6
# with  **2 - better a little
#max_score = 0.699533440786 max_index= 0 best_n_estimators= 200 best_max_depth= 6 !!!!!!!!!!!!
# original 
# max_score = 0.698382534947 max_index= 0 best_n_estimators= 200 best_max_depth= 6

import math
def modify_dayaframe_02(parDataFrame):
    print 'modify_dayaframe_02 (Before). parDataFrame', parDataFrame.shape
    power_value=2
    toPower=['r1_gold','r2_gold','r3_gold','r4_gold','r5_gold','r_gold', \
             'd1_gold','d2_gold','d3_gold','d4_gold','d5_gold','d_gold',\
            
             'r1_level','r2_level','r3_level','r4_level','r5_level','r_level', \
             'd1_level','d2_level','d3_level','d4_level','d5_level','d_level',\
            
             'r1_xp','r2_xp','r3_xp','r4_xp','r5_xp', 'r_xp', \
             'd1_xp','d2_xp','d3_xp','d4_xp','d5_xp', 'd_xp', \
            
             'r1_lh','r2_lh','r3_lh','r4_lh','r5_lh', 'r_lh', \
             'd1_lh','d2_lh','d3_lh','d4_lh','d5_lh','d_lh', \
            
             'r1_kills','r2_kills','r3_kills','r4_kills','r5_kills', 'r_kills', \
             'd1_kills','d2_kills','d3_kills','d4_kills','d5_kills', 'd_kills',\
            
             'r1_items','r2_items','r3_items','r4_items','r5_items','r_items', \
             'd1_items','d2_items','d3_items','d4_items','d5_items','d_items' \
            ]

    for index in range(len(toPower)):
        pow_field(toPower[index], power_value,parDataFrame)
        #parDataFrame[powered[index]] = \
        #    parDataFrame.apply(lambda row: math.sqrt(row[toPower[index]]), axis=1)
        #parDataFrame[powered[index]] = \
        #    parDataFrame.apply(lambda row: row[toPower[index]]**2, axis=1)
    rezDataFrame=parDataFrame
    # rezDataFrame=parDataFrame.drop( toPower, axis=1)
    print 'modify_dayaframe_02 (After). rezDataFrame', rezDataFrame.shape
    return rezDataFrame

def pow_field(fld_name, mx_pwr,parDataFrame):
    for p in xrange(mx_pwr-1):
        parDataFrame[fld_name+'pwr'+str(p+2)] = \
            parDataFrame.apply(lambda row: math.pow(row[fld_name], p+2), axis=1)

# Best usage is - add new field and not remove sources
# add summ of gold R and D fields
# === for log_regression: Gold only
# 0.703771390136  remove original gold - 
# 0.723777307057 add summ for gold R and D and remove original gold 
# 0.729734140515 add summ for gold only R and D  
# === for log_regression: gold..items only
# 0.742759042342 add summ for gold..items R and D  
# 0.716864897388 - original   (Better !!!!!!!!!!!)
# === for Gradient Boosting: 
import math
def modify_dayaframe_03(parDataFrame):
    print 'modify_dayaframe_03 (Before). parDataFrame', parDataFrame.shape
    add_sum_field('r', '_gold', parDataFrame)
    add_sum_field('d', '_gold', parDataFrame)

    add_sum_field('r', '_level', parDataFrame)
    add_sum_field('d', '_level', parDataFrame)

    add_sum_field('r', '_xp', parDataFrame)
    add_sum_field('d', '_xp', parDataFrame)


    add_sum_field('r', '_lh', parDataFrame)
    add_sum_field('d', '_lh', parDataFrame)

    add_sum_field('r', '_kills', parDataFrame)
    add_sum_field('d', '_kills', parDataFrame)

    add_sum_field('r', '_deaths', parDataFrame)
    add_sum_field('d', '_deaths', parDataFrame)

    add_sum_field('r', '_items', parDataFrame)
    add_sum_field('d', '_items', parDataFrame)


    '''
    toPower=['r1_gold','r2_gold','r3_gold','r4_gold','r5_gold', \
             'd1_gold','d2_gold','d3_gold','d4_gold','d5_gold']
    powered=['r1_gold2','r2_gold2','r3_gold2','r4_gold2','r5_gold2', \
             'd1_gold2','d2_gold2','d3_gold2','d4_gold2','d5_gold2']
    parDataFrame['r_gold'] = parDataFrame['r1_gold']+ parDataFrame['r2_gold']+ parDataFrame['r3_gold'] + \
        parDataFrame['r4_gold']+parDataFrame['r5_gold']
    parDataFrame['d_gold'] = parDataFrame['d1_gold']+ parDataFrame['d2_gold']+ parDataFrame['d3_gold'] + \
        parDataFrame['d4_gold']+parDataFrame['d5_gold']
    '''
    rezDataFrame=parDataFrame
    # rezDataFrame=parDataFrame.drop( toPower, axis=1)
    print 'modify_dayaframe_03 (After). rezDataFrame', rezDataFrame.shape
    return rezDataFrame

# add_sum_field('r', '_gold', parDataFrame)
def add_sum_field(r_d, fd_name, parDataFrame):
    parDataFrame[r_d+fd_name] = parDataFrame[r_d+'1'+fd_name]+ parDataFrame[r_d+'2'+fd_name]+ parDataFrame[r_d+'3'+fd_name] + \
        parDataFrame[r_d+'4'+fd_name]+parDataFrame[r_d+'5'+fd_name]

'''
        'r%d_hero' % (p+1)]
            X_pick[i, theX.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
            X_pick[i, theX.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1
'''    
# 
from sklearn.tree import DecisionTreeClassifier
def calculate_importance(theX, theY):
    clf = DecisionTreeClassifier(random_state=241)
    clf.fit(theX, theY)
    importances = clf.feature_importances_
    print importances
    print importances.shape
    

# Some draft code

In [ ]:
# prepare data for gradient boosting    ?????????????????????
import pandas
def prepare_for_boosting(fileName, isTest):
    # Считайте таблицу с признаками из файла features.csv с помощью кода, приведенного выше. 

    features = pandas.read_csv(fileName, index_col='match_id')
    print features.head()
    print features.info()
    
    if (isTest):        
        match_info = features
    else:    
        match_info, y = remove_result_columns(features)
        
    X_removed_heroes_rooms = remove_initial_heroes_and_lobbies(match_info)
    X = replace_skpped_values_with_0(X_removed_heroes_rooms)

    xxx_heroes= words_bag_of_heroes(match_info)
    print type(xxx_heroes)
    print xxx_heroes.shape

    xxx_rooms= words_bag_of_rooms(match_info)
    print type(xxx_rooms)
    print xxx_rooms.shape

    xxx_rooms_and_heroes=join_bags_rooms_and_heroes(xxx_heroes , xxx_rooms)

    X_bagged = add_word_bag(X,xxx_rooms_and_heroes)
    if (isTest):        
        return X_bagged, X
    else:    
        return X_bagged, y


In [ ]:
# Skip it ++++++++++++++++++++++
# Считаем для 30 - max_depth=6
rez_scores_local = list()
for n_estimators in [1,2]:    
    rez_scores_local.append((n_estimators, calculate_scores_for_estimators(n_estimators, x_bust , y_bust, max_depth=1)))    
rez_scores_local

In [ ]:
# check if max score is good
import sys
print type(rez_scores_local)
type(rez_scores_local[0])
max_score=-sys.maxint - 1
max_index="a"
for index in range(len(rez_scores_local)):
    if (rez_scores_local[index][1][0]>max_score):
        max_index=index
    print rez_scores_local[index][1][0]
print max_score, max_index

#   print 'Current fruit :', fruits[index]
# rez_scores_local[:]
#[1][0]

In [ ]:
from sklearn.tree import DecisionTreeClassifier
x_t, y_t, x_data_frame  = prepare_for_log_regr('/Volumes/fast64/My/MachineLearning/week07/Assignment01/data/features.csv', False)


xxxx_dataframe = replace_skpped_values_with_0(x_data_frame)
clf = DecisionTreeClassifier(random_state=241)
clf.fit(xxxx_dataframe , y_t)
importances = clf.feature_importances_


from scipy.stats import pearsonr
for index in range(len(importances)):
    print pearsonr(xxxx_dataframe[xxxx_dataframe.columns[index]] , y_t), xxxx_dataframe.columns[index] 


for index in range(len(importances)):
    print importances[index] , xxxx_dataframe.columns[index] 

    

# LogisticRegression - train

In [ ]:
x_train_data_frame[['first_blood_time_team','first_blood_time','first_blood_team']]
1

In [ ]:
print 1

In [ ]:
print 'Start' , printtime() 
x_train, y_train, x_train_data_frame,  = prepare_for_log_regr('/Volumes/fast64/My/MachineLearning/week07/Assignment01/data/features.csv', False)
ss = stdScaling_fit(x_train)
print 'Done1'
X_train_std_scal = stdScaling_transform( x_train ,ss)
print 'Done2'

log_reg_best_param = calculate_logreg_best_param(X_train_std_scal, y_train)
print 'Done3'

clf = LogisticRegression(penalty='l2', random_state=555, C=log_reg_best_param['C']) 
print 'Done4'

clf.fit(X_train_std_scal, y_train)

print 'Done5'
rez_dataframe = store_predicted_probability(clf, X_train_std_scal, x_train_data_frame, '/Volumes/fast64/My/MachineLearning/week07/Dota2_03/logreg_train_01_02_03_pow2.csv')
print 'Done6' , printtime() 

In [ ]:
## x_train, y_train, x_train_data_frame  = prepare_for_log_regr('/Volumes/fast64/My/MachineLearning/week07/Assignment01/data/features.csv', False)
ss = stdScaling_fit(x_train)
print 'Done1'
X_train_std_scal = stdScaling_transform( x_train ,ss)
print 'Done2'

log_reg_best_param = calculate_logreg_best_param(X_train_std_scal, y_train)
print 'Done3'

clf = LogisticRegression(penalty='l2', random_state=555, C=log_reg_best_param['C']) # !! replace with log_reg_best_param
print 'Done4'

clf.fit(X_train_std_scal, y_train)

print 'Done5'
rez_dataframe = store_predicted_probability(clf, X_train_std_scal, x_train_data_frame, '/Volumes/fast64/My/MachineLearning/week07/Dota2_03/logreg_train.csv')
print 'Done6'

In [ ]:
log_reg_best_param = calculate_logreg_best_param(X_std_scal)
log_reg_best_param

# LogisticRegression - test

In [ ]:
X_test, X_test_data_frame = prepare_for_log_regr('/Volumes/fast64/My/MachineLearning/week07/Assignment01/data/features_test.csv', True)
X_test_std_scal = stdScaling_transform( X_test ,ss)
print 'Done'

In [ ]:
rez_dataframe = store_predicted_probability(clf, X_test_std_scal, X_test_data_frame, '/Volumes/fast64/My/MachineLearning/week07/Dota2_03/logreg_test_01_02_03.csv')
print 'Done'

# Градиентный бустинг - all

In [2]:
def gradient_boosting_train_test(par_estimators, par_max_depth):
    # training
    print '############## gradient boosting: trainig. Estimators= ' , par_estimators[0], 'max_depth=',  par_max_depth 
    out_train_file='C:/My/Dota2/03/grad_boost_train_'+str(par_estimators[0])+'_'+str(par_max_depth)+'.csv'
    out_test_file='C:/My/Dota2/03/grad_boost_test_'+str(par_estimators[0])+'_'+str(par_max_depth)+'.csv'    
    print '---Start train' , printtime() 
    # x_train_bust, y_train_bust, x_train_data_frame_bust  = prepare_for_gradient_boosting('C:/My/Dota2/Data/features.csv', False)
    # print 'Done1'

    rez_scores = crossvalidate_estimators(par_estimators, x_train_bust, y_train_bust, max_depth=par_max_depth)
    print 'Done2', printtime()   

    clf= get_best_GradientBoostingClassifier(rez_scores)
    clf.fit(x_train_bust, y_train_bust)
    print 'Done3', printtime()   

    rez_dataframe = store_predicted_probability(clf, x_train_bust, x_train_data_frame_bust, out_train_file)
    print 'Done4', printtime()
    # test
    print '--- Start test ', printtime()  
    
    #x_test, X_test_data_frame = prepare_for_gradient_boosting('C:/My/Dota2/Data/features_test.csv', True)

    rez_dataframe = store_predicted_probability(clf, x_test, X_test_data_frame, out_test_file)

    print 'Done', printtime()  

In [3]:
print 'Start _ all', printtime()  
x_train_bust, y_train_bust, x_train_data_frame_bust  = prepare_for_gradient_boosting('C:/My/Dota2/Data/features.csv', False)
print '--Done prepare train', printtime()  
x_test, X_test_data_frame = prepare_for_gradient_boosting('C:/My/Dota2/Data/features_test.csv', True)
print '--Done prepare test', printtime()  
    
    
gradient_boosting_train_test([120], 4)
gradient_boosting_train_test([120], 5)
gradient_boosting_train_test([240], 6)
gradient_boosting_train_test([480], 6)
gradient_boosting_train_test([800], 6)

print 'Done _ all', printtime()  


Start _ all 2016-04-29 04:59:00.062000
modify_dayaframe_01 (Before). parDataFrame (97230, 108)
modify_dayaframe_01 (After). parDataFrame (97230, 107)
modify_dayaframe_03 (Before). parDataFrame (97230, 107)
modify_dayaframe_03 (After). rezDataFrame (97230, 121)
modify_dayaframe_02 (Before). parDataFrame (97230, 121)
modify_dayaframe_02 (After). rezDataFrame (97230, 193)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 97230 entries, 0 to 114406
Columns: 193 entries, start_time to d_itemspwr2
dtypes: float64(83), int64(110)
memory usage: 143.9 MB
None
remove_result_columns (Before): shape (97230, 193)
remove_result_columns (After): shape (97230, 187)
Before
NaNs = 173534
not NaNs = 18008476
after
NaNs = 0
not NaNs = 18182010
--Done prepare train 2016-04-29 05:03:27.581000
modify_dayaframe_01 (Before). parDataFrame (17177, 102)
modify_dayaframe_01 (After). parDataFrame (17177, 101)
modify_dayaframe_03 (Before). parDataFrame (17177, 101)
modify_dayaframe_03 (After). rezDataFrame (17177, 1

# Градиентный бустинг - train


In [ ]:
print 'Start' , printtime() 
x_train_bust, y_train_bust, x_train_data_frame_bust  = prepare_for_gradient_boosting('/Volumes/fast64/My/MachineLearning/week07/Assignment01/data/features.csv', False)
print 'Done1'

rez_scores = crossvalidate_estimators([50], x_train_bust, y_train_bust, max_depth=6)
print 'Done2', printtime()   

clf= get_best_GradientBoostingClassifier(rez_scores)
clf.fit(x_train_bust, y_train_bust)
print 'Done3', printtime()   

rez_dataframe = store_predicted_probability(clf, x_train_bust, x_train_data_frame_bust, '/Volumes/fast64/My/MachineLearning/week07/Dota2_03/grad_boost_train_200_6.csv')
print 'Done4', printtime()

In [ ]:
x_bust , y_bust, x_bust_dataframe = prepare_for_log_regr('/Volumes/fast64/My/MachineLearning/week07/Assignment01/data/features.csv', False)
print 'Done'

In [ ]:

# Считаем для 200 - max_depth=6
rez_scores_6 = list()
for n_estimators in [200]:    
    rez_scores_6.append((n_estimators, calculate_scores_for_estimators(n_estimators, x_bust , y_bust, max_depth=6)))    
rez_scores_6

In [ ]:
rez_scores = crossvalidate_estimators([300], x_bust, y_bust, max_depth=8)
clf= get_best_GradientBoostingClassifier(rez_scores)
clf.fit(x_bust, y_bust)
rez_dataframe = store_predicted_probability(clf, x_bust, match_info, '/Volumes/fast64/My/MachineLearning/week07/Dota2_03/grad_bust_test_300_8.csv')
print 'Done'


In [ ]:
rez_scores = crossvalidate_estimators([200], x_bust, y_bust, max_depth=6)
clf= get_best_GradientBoostingClassifier(rez_scores)
#clf.fit(x_bust, y_bust)
#rez_dataframe = store_predicted_probability(clf, x_bust, x_bust_dataframe, '/Volumes/fast64/My/MachineLearning/week07/Dota2_03/grad_bust_train_200_6.csv')
print 'Done', printtime()     


In [ ]:
rez_dataframe = store_predicted_probability(clf, x_bust, x_bust_dataframe, '/Volumes/fast64/My/MachineLearning/week07/Dota2_03/grad_bust_train_200_6.csv')
print 'Done', printtime()     


In [ ]:
rez_dataframe = store_predicted_probability(clf, x_bust, x_bust_dataframe, '/Volumes/fast64/My/MachineLearning/week07/Dota2_03/grad_bust_train_200_6.csv')
print 'Done'

# Градиентный бустинг - test


In [ ]:
print 'Start', printtime()  
x_test, X_test_data_frame = prepare_for_gradient_boosting('/Volumes/fast64/My/MachineLearning/week07/Assignment01/data/features_test.csv', True)

rez_dataframe = store_predicted_probability(clf, x_test, X_test_data_frame, '/Volumes/fast64/My/MachineLearning/week07/Dota2_03/grad_bust_test_200_6.csv')
print 'Done', printtime()  


# Сливаем результаты в один

In [ ]:
# сливаем массив результирующих csv в один

def join_all_results(files, rez_file_name):
    src_list= list()
    for fl in files:    
        fl_content = pandas.read_csv(fl, index_col='match_id')
        print fl_content.shape
        src_list.append(fl_content)

    lngth = len(src_list[0].index)
    rez = [0] * lngth

    src_num = len(src_list)

    for sss in src_list:
        rez = rez + sss['radiant_win']

    rez =rez/src_num

    test_check = {
        'match_id': src_list[0].index, 
        'radiant_win': rez
    }

    test_check_dframe = pandas.DataFrame.from_dict(test_check)
    test_check_dframe.set_index('match_id')

    test_check_dframe.to_csv(rez_file_name, index=False, columns=['match_id', 'radiant_win'])
    print 'done join_all_results'


In [ ]:
files = ['/Volumes/fast64/My/MachineLearning/week07/Dota2_03/grad_bust_test_300_8.csv', 
        '/Volumes/fast64/My/MachineLearning/week07/Dota2_03/logreg_test.csv']
rez_file_name = '/Volumes/fast64/My/MachineLearning/week07/Dota2_03/summary_test1.csv'
join_all_results(files, rez_file_name)

# Старое  --------
# Старое  -------- 
# Старое  -------- 


# Подход 1: градиентный бустинг "в лоб"

### Data preparation

<font color='blue'>
Проверьте выборку на наличие пропусков с помощью функции count(), которая для каждого столбца показывает число заполненных значений. Много ли пропусков в данных? Запишите названия признаков, имеющих пропуски, и попробуйте для любых двух из них дать обоснование, почему их значения могут быть пропущены.
</font>

In [ ]:
match_info_count = match_info.count()
missing_data_columns_info = match_info_count[match_info_count != match_info_count.max()]

print "Q: Много ли пропусков в данных?"
print "A: ",len(missing_data_columns_info),"признаков имеют пропуски в данных?","\n"

print  "названия признаков, имеющих пропуски"


print missing_data_columns_info
# !!!!!!!!!!! посчитать % пропуска

<font color='green'>
Q: попробуйте для любых двух из них дать обоснование, почему их значения могут быть пропущены <br/>
A:  <br/>
поля <b>first_blood_time, first_blood_team, first_blood_player1</b><br/>
не было первой крови в первые 5 минут
</font>

In [ ]:
# проверим, что нет признаков с пропусками
X_count=X.count()
ZZZ = X_count[X_count != X_count.max()]
print "Количество признаков с пропусками после заполнения =", ZZZ.size

<font color='blue'>
Какой столбец содержит целевую переменную? Запишите его название.
</font>

<font color='green'>
Q: Какой столбец содержит целевую переменную? Запишите его название. <br/>
A:  <br/>
столбец <b>radiant_win</b><br/>
1, если победила команда Radiant, 0 — иначе
</font>

<font color='blue'>
Забудем, что в выборке есть категориальные признаки, и попробуем обучить градиентный бустинг над деревьями на имеющейся матрице "объекты-признаки". Зафиксируйте генератор разбиений для кросс-валидации по 5 блокам (KFold), не забудьте перемешать при этом выборку (shuffle=True), поскольку данные в таблице отсортированы по времени, и без перемешивания можно столкнуться с нежелательными эффектами при оценивании качества. Оцените качество градиентного бустинга (GradientBoostingClassifier) с помощью данной кросс-валидации, попробуйте при этом разное количество деревьев (как минимум протестируйте следующие значения для количества деревьев: 10, 20, 30). Долго ли настраивались классификаторы? Достигнут ли оптимум на испытанных значениях параметра n_estimators, или же качество, скорее всего, продолжит расти при дальнейшем его увеличении?
</font>

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import KFold
import datetime

def calculate_scores_for_estimators(n_estimators, max_depth=3):
    start_time = datetime.datetime.now()
    clf = GradientBoostingClassifier(n_estimators=n_estimators,
        random_state=555, max_depth=max_depth,
    )
    k_fold = KFold(len(y), n_folds=5, random_state=555, shuffle=True)
    local_scores = cross_val_score(clf, X, y, cv=k_fold, scoring='roc_auc')
    print 'n_estimators =', n_estimators, 'max_depth=', max_depth, 'Time elapsed:', datetime.datetime.now() - start_time
    return local_scores.mean(), local_scores.std()

In [ ]:
# Считаем для 10,20,30, 100, 200 - max_depth=3
rez_scores_3 = list()
for n_estimators in [10,20,30,100, 200]:    
    rez_scores_3.append((n_estimators, calculate_scores_for_estimators(n_estimators=n_estimators)))    
rez_scores_3

In [ ]:
# Считаем для 10,20,30 - max_depth=2
rez_scores_2 = list()
for n_estimators in [10,20,30]:    
    rez_scores_2.append((n_estimators, calculate_scores_for_estimators(n_estimators=n_estimators, max_depth=2)))    
rez_scores_2

In [ ]:
# Считаем для 10,20,30 - max_depth=6
rez_scores_6 = list()
for n_estimators in [10,20,30]:    
    rez_scores_6.append((n_estimators, calculate_scores_for_estimators(n_estimators=n_estimators, max_depth=6)))    
rez_scores_6

# Подход 2: логистическая регрессия

<font color='blue'>
1. Оцените качество логистической регрессии (sklearn.linear_model.LogisticRegression с L2-регуляризацией) с помощью кросс-валидации по той же схеме, которая использовалась для градиентного бустинга. Подберите при этом лучший параметр регуляризации (C). Какое наилучшее качество у вас получилось? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?</font>

In [ ]:
calculate_logreg(X_std_scal)

<font color='blue'>
4. Воспользуемся подходом "мешок слов" для кодирования информации о героях. Пусть всего в игре имеет N различных героев. Сформируем N признаков, при этом i-й будет равен нулю, если i-й герой не участвовал в матче; единице, если i-й герой играл за команду Radiant; минус единице, если i-й герой играл за команду Dire. Ниже вы можете найти код, который выполняет данной преобразование. Добавьте полученные признаки к числовым, которые вы использовали во втором пункте данного этапа.
</font>

In [ ]:
X_pick = np.zeros((X.shape[0], max(unique_heroes)))

for i, match_id in enumerate(X.index):
    for p in xrange(5):
        X_pick[i, X.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, X.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1


In [ ]:
print X_pick.shape, type(X_pick.shape)
print X_categor_removed.shape, type(X_categor_removed)

from scipy.sparse import hstack
X_categor_removed_bag_words = hstack([X_categor_removed, X_pick]).toarray()
print X_categor_removed_bag_words.shape, type(X_categor_removed_bag_words)



<font color='blue'>
5. Проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Какое получилось качество? Улучшилось ли оно? Чем вы можете это объяснить?
</font>

In [ ]:
X_categor_removed_bag_words_std_scal = StandardScaler().fit_transform(X_categor_removed_bag_words)
calculate_logreg(X_categor_removed_bag_words_std_scal)

# добавим мешок слов по комнате

In [ ]:

unique_lobbyes = np.unique(X['lobby_type'])
print unique_lobbyes
print 'unique_lobbyes.size=', unique_lobbyes.size

In [ ]:
'''id,name
-1,Invalid
0,Public matchmaking
1,Practice
2,Tournament
3,Tutorial
4,Co-op with bots
5,Team match
6,Solo Queue
7,Ranked
8,Solo Mid 1vs1
'''

In [ ]:
num_rooms = 10
X_pick_room = np.zeros([X.shape[0], num_rooms])   # !!! two (())

for i, match_id in enumerate(X.index):
    for p in xrange(10):
        X_pick_room[i, X.ix[match_id, 'lobby_type']+1] = 1

In [ ]:

print X_pick.shape
print X_pick_room.shape
X_pick_heroes_room = np.concatenate((X_pick , X_pick_room),  axis=1)
print X_pick_heroes_room.shape




In [ ]:
print X_pick_heroes_room.shape, type(X_pick_heroes_room.shape)
print X_categor_removed.shape, type(X_categor_removed)

from scipy.sparse import hstack
X_categor_removed_bag_words_hero_room = hstack([X_categor_removed, X_pick_heroes_room]).toarray()
print X_categor_removed_bag_words_hero_room.shape, type(X_categor_removed_bag_words_hero_room)


In [ ]:
ss = StandardScaler().fit(X_categor_removed_bag_words_hero_room)
ttt= ss.transform(X_categor_removed_bag_words_hero_room)
print ttt.mean(), ttt.min(), ttt.max()
X_categor_removed_bag_words_hero_room_std_scal = StandardScaler().fit_transform(X_categor_removed_bag_words_hero_room)
print X_categor_removed_bag_words_hero_room_std_scal.mean(), X_categor_removed_bag_words_hero_room_std_scal.min(), X_categor_removed_bag_words_hero_room_std_scal.max()



In [ ]:
X_categor_removed_bag_words_hero_room_std_scal = StandardScaler().fit_transform(X_categor_removed_bag_words_hero_room)
calculate_logreg(X_categor_removed_bag_words_hero_room_std_scal)

<font color='blue'>
6. Постройте предсказания вероятностей победы команды Radiant для тестовой выборки с помощью лучшей из изученных моделей (лучшей с точки зрения AUC-ROC на кросс-валидации). Убедитесь, что предсказанные вероятности адекватные — находятся на отрезке [0, 1], не совпадают между собой (т.е. что модель не получилась константной).
</font>

In [ ]:
features_test = pandas.read_csv('/Volumes/fast64/My/MachineLearning/week07/Assignment01/data/features_test.csv', index_col='match_id')
print features_test.info()
X_test = features_test.fillna(0)

X_test_categor_removed= X_test.drop( heros+ ['lobby_type'], axis=1)

test_unique_heroes = np.unique(X_test[heros])
print test_unique_heroes
print 'test_unique_heroes.size=', test_unique_heroes.size


# X_test_categor_removed_std_scal = StandardScaler().fit_transform(X_test_categor_removed)


In [ ]:
X_pick = np.zeros((X_test.shape[0], max(test_unique_heroes)))

for i, match_id in enumerate(X_test.index):
    for p in xrange(5):
        X_pick[i, X_test.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, X_test.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

In [ ]:
num_rooms = 10
X_pick_room = np.zeros([X_test.shape[0], num_rooms])   # !!! two (())

for i, match_id in enumerate(X_test.index):
    for p in xrange(10):
        X_pick_room[i, X_test.ix[match_id, 'lobby_type']+1] = 1

from scipy.sparse import hstack
X_test_pick_heroes_room = np.concatenate((X_pick , X_pick_room),  axis=1)

print X_test_pick_heroes_room.shape


In [ ]:
print X_test_pick_heroes_room.shape
print X_test_categor_removed.shape

from scipy.sparse import hstack
X_test_categor_removed_bag_words = hstack([X_test_categor_removed, X_test_pick_heroes_room]).toarray()
print X_test_categor_removed_bag_words.shape


In [ ]:
X_test_categor_removed_bag_words_std_scal = ss.transform(X_test_categor_removed_bag_words)
print X_test_categor_removed_bag_words_std_scal.mean(), X_test_categor_removed_bag_words_std_scal.min(), X_test_categor_removed_bag_words_std_scal.max()

ttt = StandardScaler().fit_transform(X_categor_removed_bag_words_hero_room)
print ttt.mean(), ttt.min(), ttt.max()



In [ ]:
clf = LogisticRegression(penalty='l2', random_state=555, C=0.005)
clf.fit(X_categor_removed_bag_words_hero_room_std_scal, y)

In [ ]:
# smoke check
rez=clf.predict_proba(X_categor_removed_bag_words_hero_room_std_scal)[:, 1]

smoke_check = {
    'match_id': X.index, 
    'radiant_win_prob': rez,
    'radiant_win_actual': y[X.index]
}

smoke_check_dframe = pandas.DataFrame.from_dict(smoke_check)

smoke_check_dframe.set_index('match_id')

num_good_predictions=0
num_bad_predictions=0
for index, row in smoke_check_dframe.iterrows():
    if (row.radiant_win_prob >= 0.5 and row.radiant_win_actual == 1 or 
       row.radiant_win_prob < 0.5 and row.radiant_win_actual == 0):
        num_good_predictions=num_good_predictions+1
    else:
        num_bad_predictions=num_bad_predictions+1
print 'num_good_predictions=', num_good_predictions, 'num_bad_predictions=', num_bad_predictions
print 'ratio=', 100.0*num_good_predictions/(num_good_predictions+num_bad_predictions)


In [ ]:
rez=clf.predict_proba(X_test_categor_removed_bag_words_std_scal)[:, 1]

print "Минимум предсказанных вероятностей =",min(rez)
print "Максимум предсказанных вероятностей =", max(rez)



In [ ]:
test_check = {
    'match_id': X_test.index, 
    'radiant_win': rez
}

test_check_dframe = pandas.DataFrame.from_dict(test_check)
test_check_dframe.set_index('match_id')

test_check_dframe.to_csv('/Volumes/fast64/My/MachineLearning/week07/Dota2_01/dota2-kaggle-01.csv', index=False, columns=['match_id', 'radiant_win'])